In [433]:
# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

## Fase 1: Exploración y Limpieza

#### 1. Exploración Inicial:

Realiza una exploración inicial de los datos para identificar posibles problemas, como valores nulos, atípicos o datos faltantes en las columnas relevantes.

Utiliza funciones de Pandas para obtener información sobre la estructura de los datos, la presencia de valores nulos y estadísticas básicas de las columnas involucradas.

Une los dos conjuntos de datos de la forma más eficiente.

## CSV 1

In [434]:
# Leer CSV 1
df1 = pd.read_csv("Files/Customer Flight Activity.csv")
df1.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [435]:
df1.tail()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
405619,999902,2018,12,0,0,0,0,0.0,0,0
405620,999911,2018,12,0,0,0,0,0.0,0,0
405621,999940,2018,12,3,0,3,1233,123.0,0,0
405622,999982,2018,12,0,0,0,0,0.0,0,0
405623,999986,2018,12,0,0,0,0,0.0,0,0


In [436]:
columnas_traducidas = {
    'Loyalty Number': 'numero_cliente',
    'Year': 'ano',
    'Month': 'mes',
    'Flights Booked': 'vuelos_reservados',
    'Flights with Companions': 'vuelos_con_acompanantes',
    'Total Flights': 'total_vuelos',
    'Distance': 'distancia',
    'Points Accumulated': 'puntos_acumulados',
    'Points Redeemed': 'puntos_canjeados',
    'Dollar Cost Points Redeemed': 'costo_en_dolares_de_los_puntos_canjeados'
}
df1.rename(columns= columnas_traducidas, inplace=True)

In [437]:
df1.shape

(405624, 10)

Vemos que inicialmente tenemos 405624 filas y 10 columnas

In [438]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   numero_cliente                            405624 non-null  int64  
 1   ano                                       405624 non-null  int64  
 2   mes                                       405624 non-null  int64  
 3   vuelos_reservados                         405624 non-null  int64  
 4   vuelos_con_acompanantes                   405624 non-null  int64  
 5   total_vuelos                              405624 non-null  int64  
 6   distancia                                 405624 non-null  int64  
 7   puntos_acumulados                         405624 non-null  float64
 8   puntos_canjeados                          405624 non-null  int64  
 9   costo_en_dolares_de_los_puntos_canjeados  405624 non-null  int64  
dtypes: float64(1), int64

Todos los datos son númericos. -->  Cambiar la columna año y mes a formato fecha.

In [439]:
df1.isna().sum()

numero_cliente                              0
ano                                         0
mes                                         0
vuelos_reservados                           0
vuelos_con_acompanantes                     0
total_vuelos                                0
distancia                                   0
puntos_acumulados                           0
puntos_canjeados                            0
costo_en_dolares_de_los_puntos_canjeados    0
dtype: int64

No hay nulos en ninguna columna 

In [440]:
df1.describe()

,numero_cliente,ano,mes,vuelos_reservados,vuelos_con_acompanantes,total_vuelos,distancia,puntos_acumulados,puntos_canjeados,costo_en_dolares_de_los_puntos_canjeados
count,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000,405624.000000
mean,550037.873084,2017.500000,6.500000,4.115052,1.031805,5.146858,1208.880059,123.692721,30.696872,2.484503
std,258935.286969,0.500001,3.452057,5.225518,2.076869,6.521227,1433.155320,146.599831,125.486049,10.150038
min,100018.000000,2017.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,326961.000000,2017.000000,3.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,550834.000000,2017.500000,6.500000,1.000000,0.000000,1.000000,488.000000,50.000000,0.000000,0.000000
75%,772194.000000,2018.000000,9.250000,8.000000,1.000000,10.000000,2336.000000,239.000000,0.000000,0.000000
max,999986.000000,2018.000000,12.000000,21.000000,11.000000,32.000000,6293.000000,676.500000,876.000000,71.000000


Podemos ver que los datos son desde de enero del 2017 a diciembre del 2018.

Vemos valores atipicos en que pueden estar afectando a las medias.Hay una desviacion estandar alta.

In [441]:
df1.duplicated().sum()

1864

Tenemos valores duplicados en las filas, vamos a verlos para decidir que hacer con ellos

In [442]:
df1[df1.duplicated(keep=False)].head()

,numero_cliente,ano,mes,vuelos_reservados,vuelos_con_acompanantes,total_vuelos,distancia,puntos_acumulados,puntos_canjeados,costo_en_dolares_de_los_puntos_canjeados
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
226,112142,2017,1,0,0,0,0,0.0,0,0
227,112142,2017,1,0,0,0,0,0.0,0,0
477,126100,2017,1,0,0,0,0,0.0,0,0


Parece que se generan filas por cliente, por mes y por año aunque el cliente no tenga vuelos. --> Hay que elimar estas filas

In [443]:
df1["numero_cliente"].duplicated().sum()

388887

In [444]:
df1[df1["numero_cliente"].duplicated(keep=False)].sort_values("numero_cliente").head()

,numero_cliente,ano,mes,vuelos_reservados,vuelos_con_acompanantes,total_vuelos,distancia,puntos_acumulados,puntos_canjeados,costo_en_dolares_de_los_puntos_canjeados
0,100018,2017,1,3,0,3,1521,152.0,0,0
388723,100018,2018,12,17,6,23,4945,494.0,0,0
185911,100018,2017,12,6,0,6,1908,190.0,0,0
101406,100018,2017,7,10,0,10,3870,387.0,0,0
287317,100018,2018,6,0,0,0,0,0.0,0,0


Tenemos duplicados en la columna de numero de cliente debido a que el mismo cliente hace varios viajes. 

## CSV 2

In [445]:
# Leer CSV 2
df2 = pd.read_csv("Files/Customer Loyalty History.csv")
df2.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [446]:
columnas_traducidas2 = {
    'Loyalty Number': 'numero_cliente',
    'Country': 'pais',
    'Province': 'provincia',
    'City': 'ciudad',
    'Postal Code': 'codigo_postal',
    'Gender': 'genero',
    'Education': 'educacion',
    'Salary': 'salario',
    'Marital Status': 'estado_civil',
    'Loyalty Card': 'tarjeta_lealtad',
    'CLV': 'valoracion_cliente',
    'Enrollment Type': 'tipo_inscripcion',
    'Enrollment Year': 'ano_inscripcion',
    'Enrollment Month': 'mes_inscripcion',
    'Cancellation Year': 'ano_cancelacion',
    'Cancellation Month': 'mes_cancelacion'
}
df2.rename(columns= columnas_traducidas2, inplace=True)

In [447]:
df2.head()

,numero_cliente,pais,provincia,ciudad,codigo_postal,genero,educacion,salario,estado_civil,tarjeta_lealtad,valoracion_cliente,tipo_inscripcion,ano_inscripcion,mes_inscripcion,ano_cancelacion,mes_cancelacion
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [448]:
df2.shape

(16737, 16)

Tenemos 16737 filas y 16 columnas

In [449]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   numero_cliente      16737 non-null  int64  
 1   pais                16737 non-null  object 
 2   provincia           16737 non-null  object 
 3   ciudad              16737 non-null  object 
 4   codigo_postal       16737 non-null  object 
 5   genero              16737 non-null  object 
 6   educacion           16737 non-null  object 
 7   salario             12499 non-null  float64
 8   estado_civil        16737 non-null  object 
 9   tarjeta_lealtad     16737 non-null  object 
 10  valoracion_cliente  16737 non-null  float64
 11  tipo_inscripcion    16737 non-null  object 
 12  ano_inscripcion     16737 non-null  int64  
 13  mes_inscripcion     16737 non-null  int64  
 14  ano_cancelacion     2067 non-null   float64
 15  mes_cancelacion     2067 non-null   float64
dtypes: f

Tenemos columnas de tipo numerico (numero_cliente, salario, valoracion_cliente, ano_inscripcion, mes_inscripcion, ano_cancelacion, mes_cancelacion) --> Cambiar las columnas de año y mes a formato fecha

In [450]:
df2.isna().sum()

numero_cliente            0
pais                      0
provincia                 0
ciudad                    0
codigo_postal             0
genero                    0
educacion                 0
salario                4238
estado_civil              0
tarjeta_lealtad           0
valoracion_cliente        0
tipo_inscripcion          0
ano_inscripcion           0
mes_inscripcion           0
ano_cancelacion       14670
mes_cancelacion       14670
dtype: int64

Tenemos valores nulos en Salario y año y mes de cancelacion -- > Valorar que hacer con los nulos de salario. Los nulos de cancelación corresponden a que no han cancelado su inscripcion. 

In [451]:
df2.isna().sum()/df2.shape[0]*100

numero_cliente         0.000000
pais                   0.000000
provincia              0.000000
ciudad                 0.000000
codigo_postal          0.000000
genero                 0.000000
educacion              0.000000
salario               25.321145
estado_civil           0.000000
tarjeta_lealtad        0.000000
valoracion_cliente     0.000000
tipo_inscripcion       0.000000
ano_inscripcion        0.000000
mes_inscripcion        0.000000
ano_cancelacion       87.650117
mes_cancelacion       87.650117
dtype: float64

Vemos los nulos en porcentaje

In [452]:
df2.describe()

,numero_cliente,salario,valoracion_cliente,ano_inscripcion,mes_inscripcion,ano_cancelacion,mes_cancelacion
count,16737.000000,12499.000000,16737.000000,16737.000000,16737.000000,2067.000000,2067.000000
mean,549735.880445,79245.609409,7988.896536,2015.253211,6.669116,2016.503145,6.962748
std,258912.132453,35008.297285,6860.982280,1.979111,3.398958,1.380743,3.455297
min,100018.000000,-58486.000000,1898.010000,2012.000000,1.000000,2013.000000,1.000000
25%,326603.000000,59246.500000,3980.840000,2014.000000,4.000000,2016.000000,4.000000
50%,550434.000000,73455.000000,5780.180000,2015.000000,7.000000,2017.000000,7.000000
75%,772019.000000,88517.500000,8940.580000,2017.000000,10.000000,2018.000000,10.000000
max,999986.000000,407228.000000,83325.380000,2018.000000,12.000000,2018.000000,12.000000


Vemos valores negativos en salario --> analizar para ver que hacer con ellos

Fechas inscripcion desde enero del 2012 a diciembre del 2018

Fecha cancelacion desde enero del 2013 a diciembre del 2018

In [453]:
df2.describe(include="O")

,pais,provincia,ciudad,codigo_postal,genero,educacion,estado_civil,tarjeta_lealtad,tipo_inscripcion
count,16737,16737,16737,16737,16737,16737,16737,16737,16737
unique,1,11,29,55,2,5,3,3,2
top,Canada,Ontario,Toronto,V6E 3D9,Female,Bachelor,Married,Star,Standard
freq,16737,5404,3351,911,8410,10475,9735,7637,15766


In [454]:
for i in df2.describe(include="O").iloc[:,4:].columns:
    print(f"{i.upper()}\n{df2[i].value_counts()}\n")

GENERO
Female    8410
Male      8327
Name: genero, dtype: int64

EDUCACION
Bachelor                10475
College                  4238
High School or Below      782
Doctor                    734
Master                    508
Name: educacion, dtype: int64

ESTADO_CIVIL
Married     9735
Single      4484
Divorced    2518
Name: estado_civil, dtype: int64

TARJETA_LEALTAD
Star      7637
Nova      5671
Aurora    3429
Name: tarjeta_lealtad, dtype: int64

TIPO_INSCRIPCION
Standard          15766
2018 Promotion      971
Name: tipo_inscripcion, dtype: int64



Todos los cliente son de Canada --> Valorar eliminar columna

In [455]:
df2.duplicated().sum()

0

No tenemos filas duplicadas

In [456]:
df2["numero_cliente"].duplicated().sum()

0

No tenemos clientes duplicados

Vamos a unir ambos csv pero para ellos nos los traeremos limpios

In [457]:
df1_limpio = pd.read_csv("Files/Customer Flight Activity_limpio.csv")
df1_limpio.shape

(401912, 12)

In [458]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   numero_cliente                            405624 non-null  int64  
 1   ano                                       405624 non-null  int64  
 2   mes                                       405624 non-null  int64  
 3   vuelos_reservados                         405624 non-null  int64  
 4   vuelos_con_acompanantes                   405624 non-null  int64  
 5   total_vuelos                              405624 non-null  int64  
 6   distancia                                 405624 non-null  int64  
 7   puntos_acumulados                         405624 non-null  float64
 8   puntos_canjeados                          405624 non-null  int64  
 9   costo_en_dolares_de_los_puntos_canjeados  405624 non-null  int64  
dtypes: float64(1), int64

In [459]:
df2_limpio = pd.read_csv("Files/Customer Loyalty History_limpio.csv")
df2_limpio.shape

(16737, 17)

In [460]:
df_unido = df1_limpio.merge(df2_limpio, on="numero_cliente", how="left")

In [461]:
df_unido.shape

(401912, 28)

In [462]:
df_unido.to_csv("Files/datos_limpio_unidos.csv", index=False)

In [463]:
df1.shape

(405624, 10)

In [464]:
df1.columns

Index(['numero_cliente', 'ano', 'mes', 'vuelos_reservados',
       'vuelos_con_acompanantes', 'total_vuelos', 'distancia',
       'puntos_acumulados', 'puntos_canjeados',
       'costo_en_dolares_de_los_puntos_canjeados'],
      dtype='object')

In [465]:
df1["numero_cliente"].nunique()

16737

In [466]:
df2[["numero_cliente" ,"ano_cancelacion"]]

,numero_cliente,ano_cancelacion
0,480934,NaN
1,549612,NaN
2,429460,2018.0
3,608370,NaN
4,530508,NaN
...,...,...
16732,823768,NaN
16733,680886,NaN
16734,776187,NaN
16735,906428,NaN


In [467]:
df2.shape

(16737, 16)

In [468]:
df2.columns

Index(['numero_cliente', 'pais', 'provincia', 'ciudad', 'codigo_postal',
       'genero', 'educacion', 'salario', 'estado_civil', 'tarjeta_lealtad',
       'valoracion_cliente', 'tipo_inscripcion', 'ano_inscripcion',
       'mes_inscripcion', 'ano_cancelacion', 'mes_cancelacion'],
      dtype='object')

In [469]:
df_unido[["numero_cliente","total_vuelos", "ano", "ano_cancelacion"]].iloc[16736:16737]

,numero_cliente,total_vuelos,ano,ano_cancelacion
16736,103731,14,2017,2099
